# Retrieval API DEMO

## Code Initialisation

In [1]:
import os
import re
import requests as r
from IPython.display import Markdown, HTML
import utils as u
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [2]:
API_HOST = 'api.dowjones.com'
AUTH_HOST = 'accounts.dowjones.com'
CLIENT_ID = os.getenv('FACTIVA_CLIENTID')
USERNAME = os.getenv('FACTIVA_USERNAME')
PASSWORD = os.getenv('FACTIVA_PASSWORD')
AUTH_URL = f"https://{AUTH_HOST}/oauth2/v1/token"
LLM_CONTEXT_SIZE = 4096

## Authentication - Generate Bearer

For details about getting the `bearer_token`, please see the `utils.py` file.

In [3]:
bearer_token = u.get_bearer_token(CLIENT_ID, USERNAME, PASSWORD, AUTH_URL)
if bearer_token:
    display(Markdown(f"**Authentication Successful**: Bearer token OK for user {USERNAME.split('@')[0]}"))
else:
    display(Markdown('**Authentication Failed**: Cannot obtain the Bearer token'))

**Authentication Successful**: Bearer token OK for user 9BAI004500-svcaccount

In [4]:
req_headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json"
}

## Factiva Retrieval API Query

In [8]:
frapi_query = {
  "data": {
    "attributes": {
      "response_limit": 3,
      "query": {
        "search_filters": [
          {
            "scope": "Language",
            "value": "en"
          }
        ],
        # "What are NASA's planned missions to the Moon in 2025, and what are their primary objectives?"
        # "Summarise the latest earnings report from Microsoft Corp"
        "value": "Summarise the latest earnings report from Microsoft Corp"
      }
    },
    "id": "GenAIRetrievalExample",
    "type": "genai-content"
  }
}


## Send Query and Receive Chunks from the Retrieval API

In [9]:
chunks_resp = r.post(f"https://{API_HOST}/content/gen-ai/retrieve", json=frapi_query, headers=req_headers)

if chunks_resp.status_code == 200:
    print('Successfully retrieved chunks')
else:
    print(f"Request Failed: {chunks_resp.json()}")

Successfully retrieved chunks


## Print Chunks

In [10]:
chunks_obj = chunks_resp.json()
for article in chunks_obj['data']:
    display(Markdown(f"### {article['attributes']['headline']['main']['text']}"))
    display(Markdown(f"**{article['meta']['source']['name']}** - {article['attributes']['publication_date']} - {article['meta']['original_doc_id']} - Lang: {article['meta']['language']['code']}"))
    display(Markdown(f"{article['attributes']['snippet']['content'][0]['text']} {article['attributes']['content'][0]['text']}"))
    display(Markdown(f"---"))

### This Magnificent Seven Tech Loses More Ground After Gaining On New Quantum Chip; Is Microsoft Stock A Buy In February?

**Investor's Business Daily** - 2025-02-25 - INVDAI0020250218el2i00105 - Lang: en

With one week left to go in the month, is Microsoft stock a buy in February? This fresh take on the long-term leader among tech stocks and growth companies pinpoints the fundamentals, technicals and fund ownership factors that are ... "As colleague Andrew Bary wrote in this space last week, Meta Platforms, Microsoft and Alphabet are estimated to spend $200 billion on AI this year, equal to about one quarter of their revenue," Randall Forsyth wrote in the "Up & Down Wall Street" column in Barron's in the Week of Feb. 17 issue. "Given China-based DeepSeek's reported ability to produce an AI model for a fraction of the cost of U.S. AI companies, this raises questions about American exceptionalism, or at least the payoff from the billions U.S. tech giants are sinking into AI investments," Forsyth added. Earnings Remain Strong In terms of EPS Rating, Microsoft ranks No. 2 in its industry group behind Adobe. Going beyond the EPS Rating, Microsoft's profit picture looks solid but not spectacular. Wall Street consensus forecasts call for earnings in the current fiscal year ending in June to rise 11% to $13.15 a share, a slight downward revision. Still, that's more than quadruple what the firm earned in fiscal 2018 ($3.88). Fiscal 2026 earnings are seen accelerating slightly, up 14% to $14.95 a share. The company's return on equity in fiscal 2025 was terrific at 37%. IBD research has found that many of the great stock market winners showed an ROE of at least 17% at the start of their moves. During recent decades, ROEs have climbed past 20% among top growth stocks. Strong profit margins have also boosted MSFT's Sales + Margins + Return on equity (SMR) Rating to a stellar A on a scale of A to E. Go to IBD Stock Checkup to see the SMR Rating of any company in the IBD database. IBD's Big Picture: 'Positive' Volatility; Also, Will These Names Outperform The Chip Sector Leader? Technical Action Today Looking at a monthly chart of Microsoft stock, the megacap tech is clearly not leading the stock market higher. Since reaching an all-time high of 468.35 in July, shares have treaded water, even as the S&P 500 and Nasdaq continued their course of higher highs and higher lows. On the positive side, Microsoft stock has gotten support at 400, a psychologically important round number for investors. Let's check out the weekly chart. It shows a eight-month consolidation pattern in the works. Microsoft stock is trading 11% off its peak. But it's also trading below its 10-week and 40-week moving averages, a no-no for any stock that is preparing to break out of a good base and launch a run of new highs. A great stock normally trades above these technical levels right before a powerful breakout to new highs — the best time to grab shares for growth investors. Who Are The Future Microsofts of The Market? Check These Key Growth Stock Lists Microsoft's Strength Vs. Rest Of Market Another way to get a handle on the technical action? Consider using IBD's unique Relative Strength Rating. Compared with all other companies in the IBD database, Microsoft stock currently gets a lowly 35 Relative Strength Rating. This means Microsoft has outperformed only 35% of all stocks over the past 12 months. Four weeks ago, it was a mediocre 58; six months ago, the RS stood at 71. MarketSurge, meanwhile, notes a 30 score for the six-month RS Rating. And the three-month rating? Also weak at 31. This means Microsoft stock has outperformed only 31% of all companies in the IBD database over the past three months. Picking the best growth stocks absolutely involves selecting those that are already outperforming their peers before they stage a new strong run. When a stock breaks out of a well-formed base, you normally want to see the Relative Strength Rating exceed 80 on a scale of 1 to 99. In some cases, when the base gets extra long (think six months or longer), you can expect the rating to be lower than 80. It's not uncommon for excellent large and megacap stocks to bust out of bases and race to new highs even though their RS ratings are in the 50s or 60s. Microsoft Stock And Institutional Sponsorship Finally, fund sponsorship remains favorable and is perhaps the strongest leg in the three-legged stool of analyzing Microsoft in terms of fundamental, technical and institutional ownership criteria.

---

###                            Microsoft Corp - Company Statement

...

---

### *Microsoft 2Q Rev $69.6B >MSFT

**Dow Jones Institutional News** - 2025-01-29 - DJDN000020250129el1t0038c - Lang: en

29 Jan 2025 16:05 ET *Microsoft 2Q EPS $3.23 >MSFT *Microsoft 2Q Rev $69.6B >MSFT 

 29 Jan 2025 16:05 ET *Microsoft 2Q EPS $3.23 >MSFT 

 29 Jan 2025 16:08 ET *Microsoft: Revenue in Productivity and Business Processes Was $29.4 Billion and Increased 14% (up 13% in Constant Currency) >MSFT 

 29 Jan 2025 16:08 ET *Microsoft: Revenue in Intelligent Cloud Was $25.5 Billion and Increased 19% >MSFT 

 29 Jan 2025 16:08 ET *Microsoft: Server Products and Cloud Services Revenue Increased 21% Driven by Azure and Other Cloud Services Revenue Growth of 31% >MSFT 

 29 Jan 2025 16:08 ET *Microsoft: This Quarter Cloud Revenue Was $40.9 Billion, Up 21% Year-Over-Year, Remain Committed to Balancing Operational Discipline With Continued Investments in Cloud and AI Infrastructure >MSFT 

 29 Jan 2025 16:10 ET *Microsoft: Innovating Across Tech Stack and Helping Customers Unlock the Full ROI of AI to Capture the Massive Opportunity Ahead >MSFT 

 29 Jan 2025 16:10 ET *Microsoft: AI Business Has Surpassed an Annual Revenue Run Rate of $13 Billion, Up 175% Year-over-Year >MSFT 

 29 Jan 2025 16:10 ET *Microsoft: Will Provide Forward-Looking Guidance in Connection With This Quarterly Earnings Announcement on Earnings Conference Call and Webcast >MSFT 

 29 Jan 2025 16:14 ET *Microsoft: 2Q Operating Income $31.7 Billion and Increased 17% YoY (Up 16% in Constant Currency) >MSFT 

 29 Jan 2025 16:20 ET *Microsoft: 2Q Microsoft 365 Commercial Products and Cloud Services Revenue Increased 15% Driven by Microsoft 365 Commercial Cloud Revenue Growth of 16% (up 15% in Constant Currency) >MSFT 

 29 Jan 2025 16:20 ET *Microsoft: Microsoft 365 Consumer Products and Cloud Services Revenue Increased 8% Driven by Microsoft 365 Consumer Cloud Revenue Growth of 8% >MSFT 

 29 Jan 2025 16:20 ET *Microsoft: LinkedIn Revenue Increased 9% >MSFT 

 29 Jan 2025 16:20 ET *Microsoft: Dynamics Products and Cloud Services Revenue Increased 15% (Up 14% in Constant Currency) Driven by Dynamics 365 Revenue Growth of 19% (up 18% in Constant Currency)>MSFT 

 29 Jan 2025 16:20 ET *Microsoft: Revenue in More Personal Computing Was $14.7 Billion and Was Relatively Unchanged >MSFT 

 29 Jan 2025 16:20 ET *Microsoft: Windows OEM and Devices Revenue Increased 4% >MSFT 

 29 Jan 2025 16:20 ET *Microsoft: Xbox Content and Services Revenue Increased 2% >MSFT 

 29 Jan 2025 16:21 ET *Microsoft: Search and News Advertising Revenue Excluding Traffic Acquisition Costs Increased 21% (Up 20% in Constant Currency) >MSFT 

 29 Jan 2025 16:21 ET *Microsoft: Returned $9.7 Billion to Shareholders in the Form of Dividends and Share Repurchases in 2Q >MSFT 

 (MORE TO FOLLOW) Dow Jones Newswires 

 January 29, 2025 16:21 ET (21:21 GMT) 

 

---

## Retrieval API Conclusion

Up to this point it the seen functionalities are connected with the Retrieval API.

However, given that this service is an intermediate component in a full-stack solution, the below Test LLM steps and the [Read Article](2_read_article.ipynb) notebook are the two complementary avenues to get a full working solution.

## Test LLM

This is a downstream step that is only illustrative on how the response generation stage can be implemented. In the below example only a few articles are used to repond the prompt in a locally running LLM. The passed request is built from the original prompt enhanced with a few retrieved articles as additional context.

The goal of using only a few articles is to improve performance as the LLM can take several minutes to summarise all returned articles given the natural limitations of a desktop hardware.

The tested LLM was Microsoft's Phi-4 with 14Bn parameters. Response generations took between 1 and 2 minutes for a context between 3K to 4K tokens.

In [11]:
prompt_text = f"{frapi_query['data']['attributes']['query']['value']}. To answer use the following information:\n\n"

chunks_obj = chunks_resp.json()['data']
process_chunks = chunks_obj[0:3]
for article in process_chunks:
    prompt_text += f"{article['attributes']['headline']['main']['text']} - {article['attributes']['publication_date']} - "
    prompt_text += f"- {article['attributes']['snippet']['content'][0]['text']} {article['attributes']['content'][0]['text']}\n------\n"


In [12]:
print(prompt_text)

Summarise the latest earnings report from Microsoft Corp. To answer use the following information:

This Magnificent Seven Tech Loses More Ground After Gaining On New Quantum Chip; Is Microsoft Stock A Buy In February? - 2025-02-25 - - With one week left to go in the month, is Microsoft stock a buy in February? This fresh take on the long-term leader among tech stocks and growth companies pinpoints the fundamentals, technicals and fund ownership factors that are ... "As colleague Andrew Bary wrote in this space last week, Meta Platforms, Microsoft and Alphabet are estimated to spend $200 billion on AI this year, equal to about one quarter of their revenue," Randall Forsyth wrote in the "Up & Down Wall Street" column in Barron's in the Week of Feb. 17 issue. "Given China-based DeepSeek's reported ability to produce an AI model for a fraction of the cost of U.S. AI companies, this raises questions about American exceptionalism, or at least the payoff from the billions U.S. tech giants ar

This is a quick check to ensure the estimated number of Tokes from the `prompt_text` are below the `LLM_CONTEXT_SIZE`.

In [13]:
len(re.split(r"\b\w+\b", prompt_text)) * 1.25 < LLM_CONTEXT_SIZE

True

LLM Payload requesting a structured response

In [14]:
prompt_payload = {
    "model": "phi-4",
    "messages": [
      {
        "role": "system",
        "content": "You are an expert news analyst that respond in a professional manner. Use only the context provided by the user to answer its request at the beginning of the prompt."
      },
      {
        "role": "user",
        "content": prompt_text
      }
    ],
    "response_format": {
      "type": "json_schema",
      "json_schema": {
        "name": "prompt_response",
        "strict": "true",
        "schema": {
          "type": "object",
          "properties": {
            "response": {
              "type": "string"
            }
          },
        "required": ["response"]
        }
      }
    },
    "temperature": 0.1,
    "max_tokens": 500,
    "stream": False
  }

The endpoint URL will depend on where the model is running. A tool like LM Studio provides details for developers on the ip address that can be used for this API request. Nonetheless, the general structure is simialar to the OpenAI API and can easily be adapted to use their service.

This is the request that can take 1+ minutes to complete.

In [19]:
summ_resp = r.post(f"http://192.168.1.11:1234/v1/chat/completions", json=prompt_payload, headers={'Content-Type': 'application/json'})

In [20]:
display(HTML(f"<h3>Response:</h3><p>{summ_resp.json()['choices'][0]['message']['content']}</p>"))